In [162]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [163]:
# version and gpu support check

cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")

print('torch version:',torch.__version__)
print('device:', device)

torch version: 1.0.1.post2
device: cpu


In [164]:
batch_size = 200
lr = 0.001
momentum = 0.5
epochs = 5
log_interval=10

In [165]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_available else {}

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)



In [166]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(1600, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1,  1600)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [167]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [168]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [169]:
model = Net().to(device)

optimizer = optim.Adam(model.parameters())

for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320352
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.727657
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.551501
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.317971
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.208506
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.221410
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.207604
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.136213
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.164689
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.118186
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.128335
Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.100502
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.072276
Train Epoch: 1 [26000/60000 (43%)]	Loss: 0.109522
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.073117
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.061522
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.146103
Train Epoch: 1 [34000/60000 (57%)]	Loss: 0.108600
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.060071
Train Epoch: 1 [38000/60000 (63%)]	Loss: 0.096569
Train Epoch